In [1]:
import yaml

In [2]:
def readyml(filename):
    with open(filename, 'r') as f:
        return yaml.load(f.read())

In [3]:
pb = readyml('seam_problem.yml')

In [4]:
pb

{'driver': {'class': 'RunOnce'},
 'root': {'class': 'Group',
  'components': [{'class': 'seamtower.seamtower.SEAMTower',
    'name': 'tower',
    'parameters': {'n_sections': 21},
    'promotes': ['*']},
   {'class': 'seamloads.seamloads.SEAMLoads',
    'name': 'loads',
    'parameters': {'n_wsp': 26},
    'promotes': ['*']},
   {'class': 'seamrotor.seamrotor.SEAMBladeStructure',
    'name': 'blade',
    'promotes': ['*']},
   {'class': 'SEAM_PowerCurve',
    'from': 'seamaero.SEAMAero',
    'name': 'power_curve',
    'parameters': {'n_wsp': 26},
    'promotes': ['*']}]}}

In [5]:
from openmdao.api import Problem, Group

In [6]:
import importlib

def load_class(full_class_string):
    """
    dynamically load a class from a string
    """

    class_data = full_class_string.split(".")
    module_path = ".".join(class_data[:-1])
    class_str = class_data[-1]

    module = importlib.import_module(module_path)
    # Finally, we retrieve the Class
    return getattr(module, class_str)

In [31]:
class FUSEDProblem(Problem):
    def __init__(self, filename):
        super(FUSEDProblem, self).__init__()
        pb = readyml(filename)
        if 'root' in pb:
            if pb['root']['class'] == 'Group':
                self.root = Group()
            if 'components' in pb['root']:
                for c in pb['root']['components']:
                    if 'parameters' not in c:
                        c['parameters'] = {}
                    self.root.add(c['name'], load_class(c['class'])(**c['parameters']), promotes=c['promotes'])

    def load_inputs(self, filename):
        inputs = readyml(filename)
        for k,v in inputs.items():
            pb[k] = v

In [32]:
pb = FUSEDProblem('seam_problem.yml')
pb.setup()
pb.load_inputs('seam_turbime_inputs.yaml')
pb.run()

##############################################
Setup: Checking for potential issues...

The following parameters have no associated unknowns:
AddWeightFactorBlade
EdgeExtDynFact
EdgeFatDynFact
F
Iref
MaxChordrR
Nsections
PMtarget
SF_blade
Slim_ext_blade
Slim_fat_blade
TIF_EDext
TIF_FLext
TIF_FLfat
WeiA_input
WeiC_input
WeibullInput
air_density
blade_density
convloss
dLoad_dU_factor_flap
dLoad_dU_factor_tower
gearloss_const
gearloss_var
genloss
hub_height
lifetime_cycles
max_Cp
max_tipspeed
max_wsp
mean_wsp
min_wsp
nSigma4fatFlap
nSigma4fatTower
n_wsp
project_lifetime
rated_power
rho_steel
rotor_diameter
safety_factor_tower
sc_frac_edge
sc_frac_flap
stress_limit_extreme_tower
stress_limit_fatigue_tower
tower_bottom_diameter
tower_top_diameter
tsr
turbulence_int
wohler_exponent_blade_flap
wohler_exponent_tower

The following components have no connections:
power_curve

No recorders have been specified, so no data will be saved.

Setup: Check complete.
####################################

/Users/pire/git/DTU-SEAM/seamloads/seamloads/SEAMLoads.py:178: RuntimeWarning: divide by zero encountered in log
  FlapStel[i] = np.exp(self.wohler_exponent_blade_flap*np.log(2 * nSigma4fat[i])) * (n1p10min[i]/600)
